<a href="https://colab.research.google.com/github/Kiyopure/streamlit-llm-app/blob/main/%E3%80%90%E4%B8%AD%E9%96%93%E8%AA%B2%E9%A1%8C_%E8%A7%A3%E7%AD%94%E7%94%A8%E3%80%91%E6%9C%AC%E6%A0%BC%E7%9A%84%E3%81%AARAG_AI%E3%82%A8%E3%83%BC%E3%82%B8%E3%82%A7%E3%83%B3%E3%83%88%E3%81%AE%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0%E9%96%8B%E7%99%BA%E3%81%AB%E3%83%88%E3%83%A9%E3%82%A4%E3%81%97%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【中間課題: 解答用】本格的なRAG/AIエージェントのシステム開発にトライしよう

## 事前準備

### 【1. OpenAI APIキーの設定】

シークレット機能でOpenAI APIのAPIキーを設定しましょう。

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY").strip()

### 【2. 必要なライブラリのインストール】

コース共通

In [ ]:
!pip install langchain==0.3.26 openai==1.91.0 langchain-community==0.3.26 httpx==0.28.1

  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached openai-1.91.0-py3-none-any.whl.metadata (26 kB)
  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
Using cached langchain-0.3.26-py3-none-any.whl (1.0 MB)
Using cached openai-1.91.0-py3-none-any.whl (735 kB)
Using cached langchain_community-0.3.26-py3-none-any.whl (2.5 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.31
    Uninstalling langchain-community-0.3.31:
      Successfully uninstalled langchain-community-0.3.31
ERROR: pip's dependency resolver does not currently take into account all the packages

中間課題用

In [ ]:
!pip install python-docx==1.2.0 docx2txt==0.9 chromadb==1.0.13 tiktoken==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.12.0
    Uninstalling tiktoken-0.12.0:
      Successfully uninstalled tiktoken-0.12.0
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.23
    Uninstalling chromadb-0.5.23:
      Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.4 requires chromadb!=0.5.4,!=0.5.5,<0.6.0,>=0.4.0, but you have chromadb 1.0.13 which is incompatible.
langchain-openai 0.3.35 requires openai<3.0.0,>=1.104.2, but you have openai 1.91.0 which is incompatible.


### 【3. Googleドライブとの接続（マウント）】

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### 【4. 外部参照先である「オンラインMTG議事録」フォルダのパス設定】

In [ ]:
dir_path = "/content/drive/MyDrive/オンラインMTG議事録"

## 【中間課題①】オンラインMTG議事録を外部参照するRAGシステムの開発

### リセット処理（動作検証用）

「データベース化済み」フォルダと「.db」フォルダ内のファイルを全て削除する処理

**<font color= "red">※余裕がある方は、まずは自力での実装にトライしてみてください</font>**

In [ ]:
import shutil
import os

def clear_complete_dir(dir_path):
    print(dir_path)
    if os.path.isdir(dir_path):
        if dir_path.split("/")[-1] == ".db":
            shutil.rmtree(dir_path)
            os.mkdir(dir_path)
            print(f"作成済みの全てのベクターストアを削除しました。")
            return

    if os.path.isdir(dir_path):
        files = os.listdir(dir_path)
        if dir_path.split("/")[-1] == "データベース化済み":
            for file in files:
                os.remove(os.path.join(dir_path, file))
            print(f"対象テーマの「データベース化済み」フォルダを空にしました: {dir_path}")
            return
        for file in files:
            clear_complete_dir(os.path.join(dir_path, file))

In [ ]:
# リセット処理の実行
clear_complete_dir(dir_path)

### ヒント

#### ヒント①（気軽に開いてOK）

RAGの外部参照先である「オンラインMTG議事録」フォルダ内の各テーマフォルダ内で、「データベース化前」フォルダ内にはあるが「データベース化済み」フォルダ内にはない議事録ファイルを読み込み、また「データベース化済み」フォルダ内にファイルをコピーする処理

**<font color= "red">※階層の深いフォルダ内のファイルを全て取得するためには、「再起処理」と呼ばれる少し複雑なコードを書く必要があります。  
実装イメージが湧く方、もしくは腕試しをしたい方は、自力での実装にトライしてみてください</font>**

In [ ]:
import os

def recursive_file_check(path, theme_docs):
    # 受け取ったパスがフォルダかどうかで条件分岐
    if os.path.isdir(path):
        # 「データベース化済み」もしくは「.db」フォルダの場合はファイル読み込みの処理が不要であるため、後続の処理をストップ
        if path.split("/")[-1] == "データベース化済み" or path.split("/")[-1] == ".db":
            return
        # フォルダ内のファイル/フォルダ名の一覧を取得
        files = os.listdir(path)
        # 一覧をループ処理
        for file in files:
            # フォルダ内のファイルもしくはフォルダのパスを渡し、再度関数を呼び出す（関数の中で同じ関数を呼び出す「再帰処理」）
            recursive_file_check(os.path.join(path, file), theme_docs)
    else:
        # 受け取ったパスがファイルの場合のみ、「読み込み」と「コピー」の処理を実行
        file_load(path, theme_docs)

In [ ]:
from docx import Document
from langchain_community.document_loaders import Docx2txtLoader

def file_load(path, theme_docs):
    # パスを「/」で分割してリスト作成
    splitted_path = path.split("/")
    # パスからテーマ名を取り出し
    dir_path = "/content/drive/MyDrive/オンラインMTG議事録"
    theme_name = ""
    for path_elem in splitted_path:
        if path_elem in os.listdir(dir_path):
            theme_name = path_elem
            break
    # 「データベース化前」フォルダ内のファイルのパスであるため、「2つ階層を上げたところまでのパス」と「データベース化済み/ファイル名」のパスを連結
    save_filename = os.path.join("/".join(path.split("/")[:-2]), f"データベース化済み/{os.path.basename(path)}")
    # 「データベース化前」フォルダ内にはあるが「データベース化済み」フォルダ内にはない議事録ファイルの場合
    if not os.path.isfile(save_filename):
        # 拡張子「.docx」のファイルを読み込み
        loader = Docx2txtLoader(path)
        doc = loader.load()
        # 議事録ファイルを「データベース化済み」フォルダ内に保存
        new_doc = Document()
        new_doc.add_paragraph(doc[0].page_content)
        new_doc.save(save_filename)
        # テーマ名をキー、取得したドキュメントを値として辞書に追加
        if theme_name in theme_docs:
            theme_docs[theme_name] += doc
        else:
            theme_docs[theme_name] = doc

In [ ]:
# 辞書「theme_docs」に、各テーマ名をキーとしてドキュメント一覧を格納する
theme_docs = {}
# 「オンラインMTG議事録」フォルダ内のファイル読み込みと「データベース化済み」フォルダへのコピーの処理を実行
recursive_file_check(dir_path, theme_docs)

In [ ]:
# 辞書の中身を確認
theme_docs

#### ヒント②（基本開かない）

ベクターストア作成の事前準備

```Markdown
- CharacterTextSplitterクラスのインスタンスを作成　（※Lesson13: Chapter3: 「Document transformers」を参照）
- OpenAIEmbeddingsクラスのインスタンスを作成　（※Lesson13: Chapter3: 「Text embedding models」を参照）
- 7つのテーマ名一覧（「営業」など）が格納されたリストを作成
```

**<font color= "red">※上記ロジックを参照し、まずは自力での実装にトライしてみてください</font>**

##### 上記ロジックのコード

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings

# CharacterTextSplitterクラスのインスタンスを作成
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# OpenAIEmbeddingsクラスのインスタンスを作成
embeddings = OpenAIEmbeddings()

In [ ]:
# 7つのテーマ名一覧（「営業」など）が格納されたリストを作成
theme_list = []
for filename in os.listdir(dir_path):
    if not filename.startswith('.'):
        theme_list.append(filename)

In [ ]:
# リストの中身を確認
theme_list

#### ヒント③（最後の手段）

ベクターストアの作成と、すでに存在する場合は読み込んでデータを追加する処理

```Markdown
- 対象のベクターストアの種類
  - 全テーマ横断のベクターストア「.all_chromadb」
  - 各テーマごとのベクターストア「.テーマ名_chromadb」
```

**<font color= "red">※こちらのヒントには、中間課題を解くための最も重要なロジックとコードが書かれています。  
できる限り最後まで参照せず、トライしてみてください。</font>**

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
# テーマ名をキー、Retrieverを値として追加する用の空の辞書を用意
theme_retriever = {}

テーマ名（「営業」など）の一覧が格納されたリストに対してfor文を回し、  
新たにベクターストアに追加するデータが該当テーマにおいて存在する場合のみ、for文の中で以下のロジックを実装する。

```Markdown
- 「オンラインMTG議事録」フォルダから読み込んだデータの中で、テーマ名に該当するデータを取り出し、チャンク分割を行う　（※Lesson13: Chapter3: 「Document transformers」を参照）
- テーマに対応するベクターストアが「.db」フォルダ内に存在するかどうかで条件分岐
  - 存在する場合
    - 該当テーマのベクターストアを読み込む　（※Lesson13: Chapter3: 「Vector Stores」を参照）
    - 該当テーマのベクターストアにチャンク分割したドキュメントを追加する　（※自力での調査が必要）
    - 全テーマ横断のベクターストアを読み込む　（※Lesson13: Chapter3: 「Vector Stores」を参照）
    - 全テーマ横断のベクターストアにチャンク分割したドキュメントを追加する　（※自力での調査が必要）
  - 存在しない場合
    - チャンク分割したドキュメントを使い、該当テーマのベクターストアを作成　（※Lesson13: Chapter3: 「Vector Stores」を参照）
    - 全テーマ横断のベクターストアが「.db」フォルダ内に存在するかどうかで条件分岐
      - 存在する場合
        - 全テーマ横断のベクターストアを読み込む　（※Lesson13: Chapter3: 「Vector Stores」を参照）
        - 全テーマ横断のベクターストアにチャンク分割したドキュメントを追加する　（※自力での調査が必要）
      - 存在しない場合
        - チャンク分割したドキュメントを使い、全テーマ横断のベクターストアを作成　（※Lesson13: Chapter3: 「Vector Stores」を参照）
- ベクターストアからRetrieverを作成し、テーマ名をキー、Retrieverを値として辞書に追加
```

最終的に作成される辞書「theme_retriever」のデータ構造

```Markdown
{
    '全社': 「.全社_chromadb」のRetriever,
    '教育': 「.教育_chromadb」のRetriever,
    'マーケティング': 「.マーケティング_chromadb」のRetriever,
    '営業': 「.営業_chromadb」のRetriever,
    '採用': 「.採用_chromadb」のRetriever,
    '開発': 「.開発_chromadb」のRetriever,
    '顧客': 「.顧客_chromadb」のRetriever
}
```

##### 上記ロジックのコード
**<font color= "red">※まずは上記のロジックをもとに、解答にトライしてみてください。  
ただし多くの試行錯誤を行ったがどうしても実装できず、想定学習時間を大幅に超過した場合のみ、  
最後の手段として上記ロジックの解答例の以下コードを参照してください。</font>**

In [ ]:
# テーマ名（「営業」など）の一覧が格納されたリストに対してfor文を回す
for theme_name in theme_list:
    # 新たにベクターストアに追加するデータが該当テーマにおいて存在する場合のみ、ベクターストアの作成やデータ追加の処理を実行
    if theme_name in theme_docs:
        # 「オンラインMTG議事録」フォルダから読み込んだデータの中で、テーマ名に該当するデータを取り出し、チャンク分割を行う
        splitted_docs = text_splitter.split_documents(theme_docs[theme_name])
        # テーマに対応するベクターストアが「.db」フォルダ内に存在するかどうかで条件分岐
        if os.path.isdir(f"{dir_path}/.db/.{theme_name}_chromadb"):
            # 該当テーマのベクターストアを読み込む
            db = Chroma(persist_directory=f"{dir_path}/.db/.{theme_name}_chromadb", embedding_function=embeddings)
            # 該当テーマのベクターストアにチャンク分割したドキュメントを追加する
            db.add_documents(documents=splitted_docs)
            # 全テーマ横断のベクターストアを読み込む
            all_db = Chroma(persist_directory=f"{dir_path}/.db/.all_chromadb", embedding_function=embeddings)
            # 全テーマ横断のベクターストアにチャンク分割したドキュメントを追加する
            all_db.add_documents(documents=splitted_docs)
        else:
            # チャンク分割したドキュメントを使い、該当テーマのベクターストアを作成
            db = Chroma.from_documents(splitted_docs, embeddings, persist_directory=f"{dir_path}/.db/.{theme_name}_chromadb")
            # 全テーマ横断のベクターストアが「.db」フォルダ内に存在するかどうかで条件分岐
            if os.path.isdir(f"{dir_path}/.db/.all_chromadb"):
                # 全テーマ横断のベクターストアを読み込む
                all_db = Chroma(persist_directory=f"{dir_path}/.db/.all_chromadb", embedding_function=embeddings)
                # 全テーマ横断のベクターストアにチャンク分割したドキュメントを追加する
                all_db.add_documents(documents=splitted_docs)
            else:
                # チャンク分割したドキュメントを使い、全テーマ横断のベクターストアを作成
                all_db = Chroma.from_documents(splitted_docs, embeddings, persist_directory=f"{dir_path}/.db/.all_chromadb")

        # ベクターストアからRetrieverを作成し、テーマ名をキー、Retrieverを値として辞書に追加
        theme_retriever[theme_name] = db.as_retriever()

In [ ]:
# テーマ名をキー、Retrieverを値として追加した辞書の中身を確認
theme_retriever

#### ヒント④（ヒント①〜③の続きのコードがどうしても書けない場合に開く）

ヒント①〜③の続きで、LLMに回答を生成させるまでのロジック

**<font color= "red">※こちらのヒントには、ヒント①〜③の続きでLLMに回答を生成させるまでのロジックが書かれています。  
ここまで全てヒントを開くと「ほぼ中間課題の解答例」となります。  
そのためヒント①〜③を開いた場合、このヒントはできる限り最後まで開かずにトライしてみてください。</font>**

##### LLMに回答を生成させるまでのロジック

```Markdown
- 全テーマ横断のベクターストア「.all_chromadb」を読み込む　（※Lesson13: Chapter3: 「Vector Stores」を参照）
- Retrieverを作成する　（※Lesson13: Chapter3: 「Retrievers」を参照）
- ChatOpenAIクラスのインスタンスを、ストリーミング出力のオプション付きで作成する　（※Lesson15: Chapter2: 「各イベントのカスタム方法」を参照）
- 会話履歴の記憶機能が搭載されたRAGのChainを作成する　（※Lesson13: Chapter4を参照）
- 会話履歴を格納するための空のリストを用意する
- ユーザー入力値を変数に格納する
- ユーザー入力値と会話履歴をもとに回答を生成する　（※Lesson13: Chapter4を参照）
- 会話履歴のリストにユーザー入力値とLLMからの回答を追加する　（※Lesson13: Chapter4を参照）
- 再度ユーザー入力値をLLMに与え、会話履歴の記憶機能が搭載されていることを確認する
```

### 解答

In [ ]:
!pip install --upgrade langchain langchain-core langchain-openai chromadb python-docx docx2txt tiktoken openai

In [ ]:
!pip install langchain==0.1.17 langchain-core==0.1.48 langchain-openai==0.1.17

In [ ]:
!pip install langchain==0.1.17 langchain-core==0.1.45 langchain-openai==0.1.17

In [ ]:
!pip install pydantic==1.10.13

In [ ]:
!pip install langchain-openai==0.1.17

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain langchain-openai langchain-community langchain-text-splitters chromadb python-docx


In [ ]:
!pip install -q tqdm


In [ ]:
!pip install langchain==0.3.26 openai==1.91.0 langchain-community==0.3.26 httpx==0.28.1 chromadb python-docx docx2txt tiktoken

In [ ]:
!pip uninstall -y langchain langchain-openai langchain-community langchain-text-splitters chromadb python-docx

In [ ]:
!pip install langchain==0.1.17 langchain-openai==0.1.17 langchain-community==0.0.32 langchain-text-splitters==0.0.1 chromadb python-docx

In [ ]:
!pip uninstall -y langchain langchain-openai langchain-community langchain-text-splitters chromadb python-docx

In [ ]:
!pip install langchain langchain-openai langchain-community langchain-text-splitters chromadb python-docx

In [ ]:
!pip install -U langchain langchain-openai langchain-community langchain-text-splitters chromadb python-docx

In [ ]:
!pip install -q langchain-chroma

In [ ]:
# ==============================
# セクション1: ライブラリ & API設定
# ==============================
!pip uninstall -y openai langchain langchain-core langchain-openai langchain-community langchain-chroma chromadb
!pip install -q "openai>=1.104.2,<2.0.0"
!pip install -q "langchain==0.3.*" "langchain-openai==0.3.*" "langchain-community==0.3.*" \
                 "langchain-chroma==0.1.*" "chromadb==0.5.*" "tiktoken" "docx2txt" tqdm

import os, shutil
from tqdm import tqdm
from docx import Document
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY").strip()

from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

# ==============================
# セクション2: 初期設定パス
# ==============================
dir_path = "/content/drive/MyDrive/オンラインMTG議事録"
db_path = os.path.join(dir_path, ".db")
done_path = os.path.join(dir_path, "データベース化済み")
os.makedirs(dir_path, exist_ok=True)

# ==============================
# セクション3: フォルダリセット関数
# ==============================
def reset_folders():
    for path in [done_path, db_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
            print(f"🗑️ 削除: {path}")
        os.makedirs(path, exist_ok=True)
        print(f"📁 再作成: {path}")
    print("✅ リセット完了\n")

# ==============================
# セクション4: docx探索・読み込み
# ==============================
def recursive_file_check(path, theme_docs):
    if os.path.isdir(path):
        if os.path.basename(path) in ["データベース化済み", ".db"]:
            return
        for f in os.listdir(path):
            recursive_file_check(os.path.join(path, f), theme_docs)
    else:
        file_load(path, theme_docs)

def file_load(path, theme_docs):
    if not path.endswith(".docx"):
        return
    theme_name = None
    for elem in path.split("/"):
        if elem in os.listdir(dir_path):
            theme_name = elem
            break
    save_filename = os.path.join(done_path, os.path.relpath(path, dir_path))
    os.makedirs(os.path.dirname(save_filename), exist_ok=True)
    if not os.path.isfile(save_filename):
        loader = Docx2txtLoader(path)
        doc = loader.load()
        new_doc = Document()
        new_doc.add_paragraph(doc[0].page_content)
        new_doc.save(save_filename)
        theme_docs.setdefault(theme_name or "未分類", []).extend(doc)
        print(f"🆕 コピー＆読み込み: {os.path.basename(path)} → {save_filename}")
    else:
        print(f"⏭️ スキップ: {os.path.basename(path)}（既に処理済み）")

# ==============================
# セクション5: ベクターストア構築
# ==============================
def build_vectorstores(theme_docs):
    embeddings = OpenAIEmbeddings()
    splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    all_db_path = os.path.join(db_path, ".all_chromadb")
    all_docs = []

    for theme_name, docs in tqdm(theme_docs.items(), desc="テーマ処理中"):
        split_docs = splitter.split_documents(docs)
        theme_db_path = os.path.join(db_path, f".{theme_name}_chromadb")
        if os.path.isdir(theme_db_path):
            db = Chroma(persist_directory=theme_db_path, embedding_function=embeddings)
            db.add_documents(split_docs)
        else:
            db = Chroma.from_documents(split_docs, embeddings, persist_directory=theme_db_path)
        all_docs.extend(split_docs)

    if all_docs:
        if os.path.isdir(all_db_path):
            db = Chroma(persist_directory=all_db_path, embedding_function=embeddings)
            db.add_documents(all_docs)
        else:
            db = Chroma.from_documents(all_docs, embeddings, persist_directory=all_db_path)
        print("\n🌐 全テーマ横断ベクターストア更新完了")
        return db.as_retriever()
    else:
        print("⚠️ docx ファイルが見つかりません。ベクターストアは作成されません。")
        return None

# ==============================
# セクション6: 会話履歴付きRAGチャット
# ==============================
def chat_loop(retriever):
    if retriever is None:
        print("⚠️ retriever が存在しないため質問に回答できません。")
        return

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)
    prompt = ChatPromptTemplate.from_template(
        "以下の文脈を参考に質問に答えてください。\n\n{context}\n\n質問: {input}"
    )
    combine_chain = create_stuff_documents_chain(llm, prompt)
    qa_chain = create_retrieval_chain(retriever, combine_chain)

    conversation_history = []

    while True:
        user_input = input("\n💬 あなた: ")
        if user_input.lower() in ["exit", "quit"]:
            print("チャットを終了します。")
            break

        response = qa_chain.invoke({"input": user_input})
        answer = response["answer"]
        print(f"\n🤖 回答: {answer}")

        conversation_history.append({"user": user_input, "bot": answer})

# ==============================
# セクション7: 実行フロー
# ==============================
def main():
    print("🚀 RAGシステム起動中...\n")
    reset_folders()
    theme_docs = {}
    recursive_file_check(dir_path, theme_docs)
    print(f"\n📂 テーマ読み込み完了: {list(theme_docs.keys())}\n")
    retriever = build_vectorstores(theme_docs)
    if retriever is None:
        print("RAG システムを実行できません。docx ファイルを確認してください。")
        return
    chat_loop(retriever)

main()



Found existing installation: openai 1.109.1
Uninstalling openai-1.109.1:
  Successfully uninstalled openai-1.109.1
Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79
Found existing installation: langchain-openai 0.3.35
Uninstalling langchain-openai-0.3.35:
  Successfully uninstalled langchain-openai-0.3.35
Found existing installation: langchain-community 0.3.31
Uninstalling langchain-community-0.3.31:
  Successfully uninstalled langchain-community-0.3.31
Found existing installation: langchain-chroma 0.1.4
Uninstalling langchain-chroma-0.1.4:
  Successfully uninstalled langchain-chroma-0.1.4
Found existing installation: chromadb 0.5.23
Uninstalling chromadb-0.5.23:
  Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the 

テーマ処理中: 100%|██████████| 8/8 [00:24<00:00,  3.02s/it]



🌐 全テーマ横断ベクターストア更新完了

🤖 回答: 前者の文脈では、佐藤優がWebサイト経由でのリード数を増加させるための戦略について話し合う場を設けています。彼は目的を再確認し、具体的な施策とその効果測定について議論を進めようとしています。中村花と田中健も目的の確認に賛成し、施策の具体化や過去のデータを参考にすることの重要性を強調しています。

後者の文脈では、田中翔が新しいプロジェクトの進捗報告と今後の展開について話し合う場を設けています。彼はユーザーからのフィードバックをもとに改善案を考えており、特にユーザーエクスペリエンスの向上に焦点を当てています。佐藤美咲と鈴木健もフィードバックを共有し、具体的な改善点について意見を出し合っています。

このように、前者はリード数の増加に向けた戦略の策定に関する議論であり、後者はプロジェクトの進捗と改善点に関する意見交換が行われています。

🤖 回答: ミーティングの中で、どのような取り組みや意見が出されましたか？また、参加者たちはどのように業績の向上に貢献しようとしていますか？

🤖 回答: 採用に関するミーティングでは、営業部門の人材ニーズを把握し、効果的な採用戦略を具体化することが目的とされています。現在の採用状況として、営業部門では3名の新卒採用を目指しており、先月の説明会には多くの学生が参加したものの、エントリーシートの提出状況が低く、現在のところエントリーが2名という課題があります。

田中由美さんは、学生が興味を持つためには魅力的なコンテンツが必要で、会社の雰囲気や業務内容をわかりやすく伝えることが重要だと指摘しています。松本亮さんは、具体的な業務内容や成長機会を伝えること、インターンシップや実習の機会を充実させることが学生の興味を引くために有効だと提案しています。

このように、採用活動においては学生に実際の業務を体験してもらうことが効果的であり、インターンシッププログラムの充実が今後の課題として挙げられています。

🤖 回答: プログラムの現状について、各担当者が報告した内容をまとめると以下のようになります。

1. **新入社員向け教育プログラム（佐藤和也）**:
   - 初回の研修が終了し、参加者からのフィードバックは好評。
   - ただし、実践的な部分にもっと時間を割いてほしいという意見があ

In [ ]:
import os
import shutil
from google.colab import drive

# Check if the mount point exists and is not empty, then clean it up
mount_point = '/content/drive'
if os.path.exists(mount_point) and os.path.isdir(mount_point):
    if os.listdir(mount_point):
        print(f"Cleaning mount point: {mount_point}")
        try:
            shutil.rmtree(mount_point)
            os.makedirs(mount_point)
            print(f"Mount point cleaned: {mount_point}")
        except Exception as e:
            print(f"Error cleaning mount point: {e}")
            # Depending on the error, you might want to stop or try mounting anyway
            pass # Or raise e if you want to stop on cleanup failure

# Attempt to mount Google Drive
try:
    drive.mount(mount_point)
except ValueError as e:
    print(f"Mounting failed: {e}")
    print("Please check if the mount point is in use or if there are persistent issues.")
except Exception as e:
    print(f"An unexpected error occurred during mounting: {e}")

Cleaning mount point: /content/drive
Mount point cleaned: /content/drive
Mounted at /content/drive


## 【中間課題②】作成したRAGシステムにAIエージェント機能を搭載する

### 解答

In [ ]:
#必要なライブラリインストール
!pip install -q "langchain==0.3.*" "langchain-community==0.3.*" "langchain-openai==0.3.*" \
                "langchain-chroma==0.1.*" "chromadb==0.5.*" "tiktoken" "docx2txt" tqdm
#OpenAIのカギ（APIキー）を読み込み
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY").strip()


# 必要ライブラリ
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

import os, json
from tqdm import tqdm

# 会議メモのフォルダの場所を設定
dir_path = "/content/drive/MyDrive/オンラインMTG議事録"
db_path = os.path.join(dir_path, ".db")

# テーマ一覧
THEMES = ["営業", "マーケティング", "採用", "開発", "教育", "全社", "顧客"]

# 共通の LLM / embeddings作成
embeddings = OpenAIEmbeddings()
llm_for_chains = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# テーマ別のデータを読み込む関数
def load_retriever_for_theme(theme_name):
    theme_db_dir = os.path.join(db_path, f".{theme_name}_chromadb")
    if os.path.isdir(theme_db_dir):
        db = Chroma(persist_directory=theme_db_dir, embedding_function=embeddings)
        return db.as_retriever()
    else:
        return None

#テーマごとのQAチェーン
def make_qa_chain_and_answer(retriever, query, model_name="gpt-4o-mini"):
    llm = ChatOpenAI(model_name=model_name, temperature=0)
    prompt = ChatPromptTemplate.from_template(
        "以下の文脈を参考に質問に答えてください。\n\n{context}\n\n質問: {input}"
    )
    combine_chain = create_stuff_documents_chain(llm, prompt)
    qa_chain = create_retrieval_chain(retriever, combine_chain)
    response = qa_chain.invoke({"input": query})
    answer = response.get("answer") or response.get("output_text") or str(response)
    return answer


tools = []

# テーマごとのツール作成
for theme in THEMES:
    retriever = load_retriever_for_theme(theme)
    if retriever is None:
        # 存在しない場合は無視するが、ツールは作らない。代わりにログ出力。
        print(f"⚠️ テーマ DB が見つかりません: {theme} -> Skipped")
        continue


    def make_tool_fn(theme_name, retr):
        def tool_fn(query: str):
            """
            この関数はエージェントから呼ばれます。
            query を受け、該当テーマのベクターストアのみ参照して回答を生成して返します。
            """
            try:
                answer = make_qa_chain_and_answer(retr, query)
                return f"[参照テーマ: {theme_name}]\n{answer}"
            except Exception as e:
                return f"ツール実行中にエラーが発生しました: {e}"
        return tool_fn

    tool = Tool(
        name=f"{theme}_tool",
        func=make_tool_fn(theme, retriever),
        description=f"質問が「{theme}」に関する内容の場合に使ってください。内部的に '{theme}' のベクターストアのみを参照して回答を返します。"
    )
    tools.append(tool)

#全テーマを横断するツールを作る
all_retriever_dir = os.path.join(db_path, ".all_chromadb")
if os.path.isdir(all_retriever_dir):
    db_all = Chroma(persist_directory=all_retriever_dir, embedding_function=embeddings)
    all_retriever = db_all.as_retriever()

    def all_tool_fn(query: str):
        try:
            return "[参照テーマ: 全テーマ]\n" + make_qa_chain_and_answer(all_retriever, query)
        except Exception as e:
            return f"全テーマツール実行中にエラー: {e}"

    tools.append(
        Tool(
            name="全テーマ_tool",
            func=all_tool_fn,
            description="特定テーマでない、もしくは横断検索したい場合に用いるツール。全データベース（.all_chromadb）を参照します。"
        )
    )
else:
    print("⚠️ 全テーマ用 .all_chromadb が見つかりません（フォールバックツールは未作成）")

print("作成したツール:", [t.name for t in tools])


# ツールを使いこなすエージェントを作成
agent_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
agent = initialize_agent(
    tools,
    agent_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# チャットループ（エージェントに入力を投げる）

def agent_chat_loop():
    print("\n🚀 エージェントチャット開始（終了するには 'exit' / 'quit' を入力）")
    conversation_history = []
    while True:
        user_input = input("\n💬 あなた: ").strip()
        if not user_input:
            continue
        if user_input.lower() in ["exit", "quit"]:
            print("チャットを終了します。")
            break

        # エージェントに user_input を渡す（内部でツールを選択して実行）
        try:
            result = agent.run(user_input)
        except Exception as e:
            result = f"エージェント実行中に例外が発生しました: {e}"

        print(f"\n🤖 回答:\n{result}\n")
        conversation_history.append({"user": user_input, "bot": result})

# 実行
agent_chat_loop()


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture

作成したツール: ['営業_tool', 'マーケティング_tool', '採用_tool', '開発_tool', '教育_tool', '全社_tool', '顧客_tool', '全テーマ_tool']

🚀 エージェントチャット開始（終了するには 'exit' / 'quit' を入力）


> Entering new AgentExecutor chain...
マーケティング戦略に基づく採用計画についての具体的なアクションプランを知りたいので、まずはマーケティングに関する情報を取得する必要があります。  
Action: マーケティング_tool  
Action Input: マーケティング戦略に基づく採用計画の具体的なアクションプラン  

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Observation: [参照テーマ: マーケティング]
マーケティング戦略に基づく採用計画の具体的なアクションプランを策定するためには、以下のステップを考慮することが重要です。

### 1. ターゲット層の特定
- **求める人材のプロファイル作成**: 必要なスキル、経験、価値観を明確にし、どのような人材が企業文化に合うかを定義します。
- **ターゲット層の理解**: 若手人材、中堅層、シニア層など、どの層をターゲットにするかを決定します。

### 2. 採用チャネルの選定
- **SNSの活用**: InstagramやLinkedInなど、ターゲット層に合ったSNSを選び、求人情報や企業文化を発信します。
- **求人サイトの利用**: 業界特化型の求人サイトや、一般的な求人サイトを活用して広く募集を行います。

### 3. コンテンツマーケティングの実施
- **企業の魅力を伝えるコンテンツ作成**: 社員インタビューや職場の雰囲気を伝える動画、ブログ記事を作成し、WebサイトやSNSで発信します。
- **成功事例の共有**: 過去の採用成功事例や、社員の成長ストーリーを紹介することで、求職者の興味を引きます。

### 4. インフルエンサーやブランドアンバサダーの活用
- **業界のインフルエンサーとの連携**: 影響力のある人物に企業の魅力を伝えてもらい、リーチを拡大します。
- **社員をブランドアンバサダーに**: 社員が自らのSNSで企業の魅力を発信することで、リアルな声を届けます。

### 5. 採用プロセスの最適化
- **選考フローの見直し**: 応募から内定までのプロセスをスムーズにし、候補者にとってストレスの少ない体験を提供します。
- **フィードバックの実施**: 候補者からのフィードバックを受け取り、採用プロセスの改善に活かします。

### 6. 効果測定と改善
- **KPIの設定**: 採用活動の効果を測定するための指標（応募数、内定率、採用後の定着率など）を設定します。
- **定期的なレビュー**: 採用活動の進捗を定期的にレビューし、必要に応じて戦略を見直します。

### 7. 次回のミーティングに向けた準備
- **アクションプランの整理**: 各施策の担当者を決定し、実施時期を設定します。
- 